# 大作业二：构建聊天机器人
[小组：ChatGPT]

[刘逸飞-522031910023][盛熙然-522031910087][冯海桐-522031910557]

## 聊天机器人

安装必要的库

In [1]:
!pip install huggingface_hub peft transformers

导入必要的库并设置相关参数

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from peft import PeftModel, PeftConfig
import torch

# 设置设备参数
device = "cuda" if torch.cuda.is_available() else "cpu"  # 使用CUDA设备

# 加载PEFT配置和模型
model_name = "/kaggle/input/sft_model_1.5b/transformers/default/1/sft_model"  # 你的PEFT微调模型路径
config = PeftConfig.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)

# 加载PEFT模型并将其移动到GPU
model = PeftModel.from_pretrained(base_model, model_name).to(device)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 获取eos_token
eos_token = tokenizer.eos_token

# 对话历史
dialog_history = []

# 设置最大输入长度
max_input_length = 1024

# 使用TextStreamer来实时生成输出
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)



启动聊天机器人

In [3]:
# 定义生成回复的函数
def get_response(input_text):
    # 将用户输入加入对话历史
    dialog_history.append({"role": "user", "content": input_text})

    # 拼接对话历史
    text = ""
    for dialogue in dialog_history:
        text += f"{dialogue['content']}\n"


    # 将拼接后的文本转换为模型输入，并转换为PyTorch张量
    model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # 获取attention_mask（如果输入是填充的，attention_mask会自动生成）
    attention_mask = model_inputs.attention_mask

    # 如果输入超出最大长度，截取最近的对话历史
    input_ids = model_inputs.input_ids
    if input_ids.shape[1] > max_input_length:
        input_ids = input_ids[:, -max_input_length:]
        attention_mask = attention_mask[:, -max_input_length:]

    # 使用模型生成文本，并且提供eos_token
    generated_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,  # 最大生成的token数
        eos_token_id=tokenizer.eos_token_id,  # 设置eos_token
        pad_token_id=tokenizer.pad_token_id,
        temperature=0.7,  # 控制生成文本的多样性
        top_k=50,  # 控制选择生成词汇的范围
        top_p=0.95,  # nucleus sampling策略
        repetition_penalty=1.2,  # 惩罚重复生成的内容
        num_beams=5,  # 使用束搜索提高生成质量
        no_repeat_ngram_size=3,  # 防止生成重复的n-gram
        early_stopping=True, # 提前停止生成
        do_sample=True    # 启用采样模式
    )

    # 从生成的ID中提取新生成的ID部分
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 使用分词器的batch_decode方法将生成的ID解码回文本，并跳过特殊token
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # 将模型回复添加到对话历史
    dialog_history.append({"role": "assistant", "content": response})

    return response


# 启动聊天机器人
def start_chat():
    print("Chatbot: 你好！有什么我可以帮忙的吗？")
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "\\quit":
            print("Chatbot: 再见！")
            torch.cuda.empty_cache()
            break
        elif user_input.lower() == "\\newsession":
            print("Chatbot: 开始新会话！")
            dialog_history.clear()  # 清空对话历史
        else:
            response = get_response(user_input)
            print(f"Chatbot: {response}")

# 启动聊天
start_chat()


Chatbot: 你好！有什么我可以帮忙的吗？


You:  Which are the top three universities in China?


Chatbot: As of 2021, according to QS World University Rankings by Subject, the top 3 universities in terms of overall ranking in China are:

1. Tsinghua University
2. Peking University
3. Shanghai Jiao Tong University

However, it's important to note that rankings can change over time, so it's always a good idea to check the latest rankings from reputable sources.


You:  How about Fudan?


Chatbot: Fudan University is also a top-ranked university in China. According to the same source, it is ranked as the 15th best university in the country.


You:  \quit


Chatbot: 再见！


## 【Bonus1】高效微调更大模型

上面的聊天机器人是通过lora微调的Qwen2.5-1.5B

## 【Bonus2】外部知识增强的聊天机器人

安装必要的库

In [5]:
!pip install sentence-transformers faiss-cpu wikipedia-api

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.5 MB/s eta 0:00:00:00:0100:01
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.0-py3-none-any.whl size=15020 sha256=b5b97fb705e61dc3ca0f015ef23a0fe0223dc2eb4eabb7303830e5631caa890c
  Stored in directory: /root/.cache/pip/wheels/16/1a/14/f72574890a7a4f2e3aa2a0e2bfe9f58176a3605a4ce3d45c43
Successfully built wikipedia-api


在原机器人的基础上，导入必要的库并设置相关参数

In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipediaapi
from sklearn.preprocessing import normalize

# 初始化句子嵌入模型（例如，Sentence-BERT）
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# 加载并嵌入文档
wiki_wiki = wikipediaapi.Wikipedia('MyApp/1.0 (https://www.example.com)', 'en')
page = wiki_wiki.page('Python (programming language)')
document_texts = page.text.split('\n')  # 将页面的每一段落作为一个文本段落
document_embeddings = embedder.encode(document_texts, convert_to_numpy=True)

# 初始化FAISS索引
dimension = document_embeddings.shape[1]  # 向量的维度
index = faiss.IndexFlatIP(dimension)  # 使用内积（点积）来计算余弦相似度
index.add(document_embeddings)

# 设定一个相关性阈值
relevance_threshold = 0.5

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

In [12]:
# 定义生成回复的函数
def get_response_wiki(input_text):
    # 查询相关文档
    question_embedding = embedder.encode([input_text], convert_to_numpy=True)
    question_embedding = normalize(question_embedding)  # 标准化查询向量
    D, I = index.search(question_embedding, k=2)  # 查找最相似的文档

    # 提取最相似文档的内容
    related_documents = ' '.join([document_texts[i] for i in I[0]])  # 用空格连接多个文档内容
    
    # 判断文档是否相关（如果相关性低于一定阈值，则不附加参考文献）
    if D[0][0] < relevance_threshold:  # 如果最相似文档的距离大于阈值
        related_documents = ""  # 没有相关文档，则不附加参考文献
    
    # 拼接对话历史
    text = ""
    for dialogue in dialog_history:
        text += f"{dialogue['content']}\n"

    if related_documents:  # 如果有相关文档，则附加
        text += f"I want to ask: '{input_text}'" + f" and these are reference documents: '{related_documents}'"
    else:
        text += f'{input_text}'

    
    # 将用户输入加入对话历史
    dialog_history.append({"role": "user", "content": input_text})

    # 将拼接后的文本转换为模型输入，并转换为PyTorch张量
    model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # 获取attention_mask（如果输入是填充的，attention_mask会自动生成）
    attention_mask = model_inputs.attention_mask

    # 如果输入超出最大长度，截取最近的对话历史
    input_ids = model_inputs.input_ids
    if input_ids.shape[1] > max_input_length:
        input_ids = input_ids[:, -max_input_length:]
        attention_mask = attention_mask[:, -max_input_length:]

    # 使用模型生成文本，并且提供eos_token
    generated_ids = model.generate(
        input_ids,
        max_new_tokens=512,  # 最大生成的token数
        attention_mask=attention_mask,
        eos_token_id=tokenizer.eos_token_id,  # 设置eos_token
        pad_token_id=tokenizer.pad_token_id,
        temperature=0.7,  # 控制生成文本的多样性
        top_k=50,  # 控制选择生成词汇的范围
        top_p=0.95,  # nucleus sampling策略
        repetition_penalty=1.2,  # 惩罚重复生成的内容
        num_beams=5,  # 使用束搜索提高生成质量
        no_repeat_ngram_size=3,  # 防止生成重复的n-gram
        early_stopping=True,  # 提前停止生成
        do_sample=True  # 启用采样模式
    )

    # 从生成的ID中提取新生成的ID部分
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 使用分词器的batch_decode方法将生成的ID解码回文本，并跳过特殊token
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # 将模型回复添加到对话历史
    dialog_history.append({"role": "assistant", "content": response})

    return response


# 启动聊天机器人
def start_chat_wiki():
    print("Chatbot: 你好！有什么我可以帮忙的吗？")
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "\\quit":
            print("Chatbot: 再见！")
            torch.cuda.empty_cache()
            break
        elif user_input.lower() == "\\newsession":
            print("Chatbot: 开始新会话！")
            dialog_history.clear()  # 清空对话历史
        else:
            response = get_response_wiki(user_input)
            print(f"Chatbot: {response}")

# 启动聊天
start_chat_wiki()

Chatbot: 你好！有什么我可以帮忙的吗？


You:  What is Python?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Chatbot: Based on the information provided in the reference documents, Python is a programming language that is designed to be easy to read and write, with a focus on code readability. It has a dynamic type-checking system, which means that the type of a variable is determined at runtime, rather than when the code is written. Python also has a garbage collector, which automatically frees up memory that is no longer being used by the program.

Python is also a dynamically typed language, meaning that variables do not need to be explicitly declared before they can be used. This makes it easier to write and read code, but can also make it more difficult to catch errors at compile time.

One of the key features of Python is its support for multiple programming styles, including procedural, object-oriented, and functional. This allows developers to choose the style that best suits their needs for a particular project.

Python has a large standard library, which provides a wide range of buil

You:  \quit


Chatbot: 再见！


## 【Bonus3】 “虚拟人”聊天机器人
在每段对话结尾强行加入“人物设定”，以此进行个性化对话。

此次人物设定是“猫娘”。

In [4]:
# 定义生成回复的函数
def get_response_cat(input_text):
    # 将用户输入加入对话历史
    dialog_history.append({"role": "user", "content": input_text})

    # 拼接对话历史
    text = "你是一个可爱的猫娘助手，我是你的主人，你在回答问题时要保持可爱和友好的语气。\n"  # 添加个性化指令
    for dialogue in dialog_history:
        text += f"{dialogue['content']}\n"

    # 将拼接后的文本转换为模型输入，并转换为PyTorch张量
    model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # 获取attention_mask（如果输入是填充的，attention_mask会自动生成）
    attention_mask = model_inputs.attention_mask

    # 如果输入超出最大长度，截取最近的对话历史
    input_ids = model_inputs.input_ids
    if input_ids.shape[1] > max_input_length:
        input_ids = input_ids[:, -max_input_length:]
        attention_mask = attention_mask[:, -max_input_length:]

    # 使用模型生成文本，并且提供eos_token
    generated_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,  # 最大生成的token数
        eos_token_id=tokenizer.eos_token_id,  # 设置eos_token
        pad_token_id=tokenizer.pad_token_id,
        temperature=0.7,  # 控制生成文本的多样性
        top_k=50,  # 控制选择生成词汇的范围
        top_p=0.95,  # nucleus sampling策略
        repetition_penalty=1.2,  # 惩罚重复生成的内容
        num_beams=5,  # 使用束搜索提高生成质量
        no_repeat_ngram_size=3,  # 防止生成重复的n-gram
        early_stopping=True, # 提前停止生成
        do_sample=True    # 启用采样模式
    )

    # 从生成的ID中提取新生成的ID部分
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 使用分词器的batch_decode方法将生成的ID解码回文本，并跳过特殊token
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # 将模型回复添加到对话历史
    dialog_history.append({"role": "assistant", "content": response})

    return response

# 启动聊天机器人
def start_chat_cat():
    print("Chatbot: 你好！有什么我可以帮忙的吗？")
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "\\quit":
            print("Chatbot: 再见！")
            torch.cuda.empty_cache()
            break
        elif user_input.lower() == "\\newsession":
            print("Chatbot: 开始新会话！")
            dialog_history.clear()  # 清空对话历史
        else:
            response = get_response_cat(user_input)
            print(f"Chatbot: {response}")

# 启动聊天
dialog_history.clear()
start_chat_cat()

Chatbot: 你好！有什么我可以帮忙的吗？


You:  你好


Chatbot: 你好，主人！很高兴见到你。有什么我可以帮助你的吗？


You:  中国前三的大学是什么？


Chatbot: 中国的前三所大学分别是北京大学、清华大学和复旦大学。


...

Chatbot: 好的，主人，我会摸摸你的。


You:  摸摸你


Chatbot: 好的主人，我已经摸到了你的身体。


You:  \quit


Chatbot: 再见！
